In [1]:
LC0_PATH =r"Lc0/lc0.exe"

In [2]:
import chess
import chess.engine
import os

In [3]:
os.path.exists(LC0_PATH)

True

In [4]:
engine = await chess.engine.popen_uci(LC0_PATH)

<UciProtocol (pid=12136)>: stderr >>        _
<UciProtocol (pid=12136)>: stderr >> |   _ | |
<UciProtocol (pid=12136)>: stderr >> |_ |_ |_| v0.31.2 built Oct 20 2024


In [ ]:
async def analyze_position_with_lc0(fen: str, nodes_limit: int = 10000, time_limit: float = None):
    """
    Analyzes a single chess position using Leela Chess Zero (Lc0).

    Args:
        fen: The FEN string of the position to analyze.
        nodes_limit: The maximum number of nodes Lc0 should explore (playouts).
                     Lc0 typically uses 'nodes' instead of 'depth' for its search limit.
        time_limit: The maximum time in seconds Lc0 should spend on analysis.

    Returns:
        A dictionary containing the analysis results (score, principal variation).
    """
    board = chess.Board(fen)
    engine = await chess.engine.popen_uci(LC0_PATH)

    try:
        # Lc0 specific options (these are common and good starting points)
        # Check Lc0's documentation for all available options
        await engine.configure({
            "Backend": "cudnn-fp16",  # Use CUDA with half-precision for speed (requires cuDNN)
                                     # Other options: "cuda", "cpu", "opencl"
            "WeightsFile": network_files[0] if network_files else "", # Lc0 usually picks newest, but explicit is good
            "Threads": 1,            # Lc0 itself is heavily GPU-parallel; 1 thread is often fine for MCTS.
                                     # More threads might be useful if you're doing heavy CPU preprocessing
                                     # or if your Lc0 build benefits from it for specific backends.
            "MinibatchSize": 2048    # Adjust based on your GPU's VRAM. Larger can be faster.
        })

        limit = chess.engine.Limit(nodes=nodes_limit)
        if time_limit:
            limit = chess.engine.Limit(time=time_limit)

        print(f"Analyzing FEN: {fen} with Lc0 (Nodes: {nodes_limit}, Time: {time_limit}s)...")
        info = await engine.analyse(board, limit=limit)

        # Lc0 returns score as "cp" (centipawns) or "mate"
        score = info["score"].white().score(mate_score=10000) # Use a large value for mate scores
        pv = [move.uci() for move in info["pv"]]

        return {"fen": fen, "score": score, "pv": pv}

    except Exception as e:
        print(f"An error occurred during Lc0 analysis: {e}")
        return {"fen": fen, "error": str(e)}
    finally:
        await engine.quit() # Always quit the engine process